In [ ]:
cd /home/rohankum/capstone/notebooks

In [ ]:
import openai, os
openai.api_key = '<add key here>'
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
import pandas as pd
pd.set_option('display.max_colwidth', None)
from joblib import Parallel, delayed
from tqdm import tqdm
import ast, random

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [ ]:
%%timeit

completion_with_backoff(model="text-davinci-003", prompt="Once upon a time,")

In [ ]:
df = pd.read_csv('../data/tail_1ksampled_alias.csv', index_col=0)
del df['answers']
df.s2 = df.s2.apply(ast.literal_eval)
df.head()

In [ ]:
def get_que_gpt3(df, cols = ['s1', 'property', 's2']):
    if 'question' in df.columns:
        print('Error: already contains que')
        return df
    
    prompt = '''generate questions:

    obama | born | hawaii => where was obama born?

    sky | color | blue => what color is the sky?

    {} | {} | {} =>'''

    api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt.format(s1, p, s2)) \
                                                                    for s1, p, s2 in tqdm(df[cols].values))
    questions = [x.choices[0].text.strip() for x in api_response]
    df['question'] = questions
    df['api_response'] = api_response
    
    return df


def get_ans_gpt3(df, question_col = 'question'):
    prompt_answer = '''answer questions:
    where was obama born? => hawaii
    what color is the sky? => blue
    {} =>'''

    api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt_answer.format(que)) \
                                                                    for que in tqdm(df[question_col]))
    answers = [x.choices[0].text.strip() for x in api_response]
    df['answer'] = answers
    df['api_response_answer'] = api_response
    
    return df

In [ ]:
df['s2_random'] = df['s2'].apply(random.choice)
df.head()

In [ ]:
df = get_que_gpt3(df, cols=['s1', 'property', 's2_random'])

In [ ]:
df = get_ans_gpt3(df)

In [ ]:
df

In [ ]:
# df.to_csv('../data/tail2.0_1ksampled_alias_with_gpt3_qa.csv')